### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.stats import spearmanr, kendalltau

### Load data

## Пункт 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.


In [2]:
with open('tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [3]:
def pickle_to_df_tournaments(dict_from_pickle, year):
    d = defaultdict(list)
    for val in dict_from_pickle.values():
        if year is not None and val['dateStart'][:4] != year:
            continue
        d['id'].append(val['id'])
        d['name'].append(val['name'])
        d['dateStart'].append(val['dateStart'])
        d['questionQty'].append(sum(val['questionQty'].values()))
    return pd.DataFrame(d)

In [4]:
train_tournaments_df = pickle_to_df_tournaments(tournaments, '2019')
test_tournaments_df = pickle_to_df_tournaments(tournaments, '2020')

### Load preprocessed data

In [5]:
train = pd.read_pickle('result_df_train.pkl')
test = pd.read_pickle('result_df_test.pkl')

In [6]:
train.shape, test.shape

((415271, 9), (108928, 9))

## Пункт 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [7]:
train = train[train['mask'].notna()]
test  = test[test['mask'].notna()]

In [8]:
players = train.player_id.unique()
tournaments = train.tournament_id.unique()

In [9]:
players_new_id_map = {k: i for i, k in enumerate(players)}
tournament_new_id_map = {k: i for i, k in enumerate(tournaments)}

In [10]:
train['player_new_id'] = train['player_id'].map(players_new_id_map)
train['tournament_new_id'] = train['tournament_id'].map(tournament_new_id_map)
test['player_new_id'] = test['player_id'].map(players_new_id_map)

In [11]:
tournaments_max_questions = train.groupby('tournament_new_id')['mask_len'].max()

In [12]:
tournament_question_start_id_dict = dict(tournaments_max_questions.cumsum() - 36)

Раздадим всем вопросам на всех турнирах уникальные ID

In [13]:
X, y = [], []
for mask, playes_id, tournament_id  in zip(train['mask'], train['player_new_id'], train['tournament_new_id']):
    for i, target in enumerate(mask):
#         assert target in ['0', '1'], f'Got incorrect mask value: {target} (tournament_id={tid}, player_id={pid})'
        if target == '1':
            target = 1
        else:
            target = 0
        question_id = tournament_question_start_id_dict[tournament_id] + i
        X.append([question_id, playes_id])
        y.append(target)
train_X, train_y = np.array(X), np.array(y)

In [14]:
question_max_token = tournaments_max_questions.cumsum().max()

In [15]:
def to_csr(X):
    x_rows = np.repeat(np.arange(X.shape[0]), 2)
    x_cols = X.copy()
    x_cols[:,1] += question_max_token + 1
    x_cols = x_cols.ravel()
    x_values = np.ones(X.shape[0] * 2, dtype=np.uint8)
    csr_x = csr_matrix((x_values, (x_rows, x_cols)), dtype=np.uint8)
    return csr_x

In [16]:
train_X_csr = to_csr(train_X)

In [17]:
model = LogisticRegression(C=5, fit_intercept=False)

In [18]:
model.fit(train_X_csr, train_y)

C:\Users\User\Anaconda3\envs\edu\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=5, fit_intercept=False)

Расставим рейтинг игроков по весу, которые выставила модель для выбранного пользователя

In [19]:
player_weights = model.coef_[0, question_max_token + 1:]

In [20]:
players_rating = {i:rating for i, rating in enumerate(player_weights)}

In [21]:
players_rating_df = train[['player_name', 'player_new_id']].drop_duplicates(subset=['player_new_id'])

In [22]:
players_rating_df['rating'] = players_rating_df['player_new_id'].map(players_rating)

In [23]:
players_rating_df = players_rating_df.sort_values(by='rating', ascending=False)

In [24]:
players_rating_df = players_rating_df.reset_index(drop=True)

Получили вот такие результаты, где индекс показывает место в рейтинге по бейзлайн модели

In [25]:
players_rating_df.head(20)

,player_name,player_new_id,rating
0,Бурак Ирина,122,3.309359
1,Саранцев Алексей,131,2.871975
2,Руссо Максим,8222,2.471941
3,Семёнова Инна,22,2.361497
4,Выменец Юрий,0,2.357075
5,Брутер Александра,5957,2.355433
6,Семушин Иван,1209,2.331661
7,Либер Александр,1,2.295025
8,Савченков Михаил,1208,2.231117
9,Вальтер Дмитрий,33098,2.212643


Для проверки на адеватность проверил рейтинг в моем списке для топа игроков https://rating.chgk.info/players.php <br>
В целом получилось, что топовые иггроки оказались на высоких позициях <br>
PS Sorry, это все модель :)

In [26]:
players_rating_df[players_rating_df['player_name'] == 'Николенко Сергей']

,player_name,player_new_id,rating
10196,Николенко Сергей,3,-0.294255


In [27]:
players_rating_df[players_rating_df['player_name'] == 'Сорожкин Артём']

,player_name,player_new_id,rating
63,Сорожкин Артём,155,1.687601


In [28]:
players_rating_df[players_rating_df['player_name'] == 'Савченков Михаил']

,player_name,player_new_id,rating
8,Савченков Михаил,1208,2.231117


In [29]:
players_rating_df[players_rating_df['player_name'] == 'Левандовский Михаил']

,player_name,player_new_id,rating
211,Левандовский Михаил,2,1.373052


In [30]:
players_rating_df[players_rating_df['player_name'] == 'Брутер Александра']

,player_name,player_new_id,rating
5,Брутер Александра,5957,2.355433


## Пункт 3
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [31]:
def get_team_rate(model, players):
    player_weights = model.coef_[0, question_max_token + 1:]
    players_rating = {i:rating for i, rating in enumerate(player_weights)}
    rates = np.array([players_rating[player_id] for player_id in players])
    return 1 - np.prod(1 - rates)

In [32]:
def get_scores(df, model):
    spearman_scores = []
    kendall_scores = []
    for _, tournament_df in df.groupby('tournament_id'):
        true_scores = []
        pred_scores = []
        for _, team_df in tournament_df.groupby('team_id'):
            team_players = team_df.player_new_id.values
            team_true_score = team_df.questionsTotal.iloc[0]
            team_pred_score = get_team_rate(model, team_players)
            true_scores.append(team_true_score)
            pred_scores.append(team_pred_score)
        spearman_scores.append(spearmanr(true_scores, pred_scores).correlation)
        kendall_scores.append(kendalltau(true_scores, pred_scores).correlation)
    return np.array(spearman_scores), np.array(kendall_scores)

In [33]:
spearman_scores, kendall_scores = get_scores(train, model)

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Усредненные результаты корреляций

In [34]:
spearman_scores[~np.isnan(spearman_scores)].mean(), kendall_scores[~np.isnan(kendall_scores)].mean()

(0.7683373991577641, 0.61227306918948)

## Пункт 4 
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики. <br>
Составим ЕМ схему на основе линейной регрессии. Скрытые переменные после Е шага будем передавать как веса на М шаге, где происходит перерасчет модели

In [35]:
def make_prediction_meta(results):
    q_tokens, team_ids, y = [], [], []
    for mask, tournament_id, team_id in zip(results['mask'], results.tournament_new_id, results.team_id):
        q_tokens.extend([tournament_question_start_id_dict[tournament_id] + i for i in range(len(mask))])
        team_ids.extend([team_id] * len(mask))  
        y.extend([True if c == '1' else False for c in mask])
    return pd.DataFrame({'question_token': q_tokens, 'team_id': team_ids, 'team_answered': y})

In [36]:
train_meta = make_prediction_meta(train)

In [37]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [38]:
def e_step(model, X, meta):
    meta = meta.copy()
    pred = model.predict(X)
    pred = sigmoid(pred)
    if len(pred.shape) == 2:
        pred = pred[:,1] if pred.shape[1] == 2 else pred.ravel()
    meta['pred'] = pred
    meta['player_fail_proba'] = 1 - pred
    team_fail_proba = meta.groupby(['question_token', 'team_id'])['player_fail_proba'].prod().rename('team_fail_proba')
    meta = meta.merge(team_fail_proba, left_on=['question_token', 'team_id'], right_index=True)
    meta['team_success_proba'] = 1 - meta['team_fail_proba']
    meta['normalized_pred'] = meta.pred / meta['team_success_proba']
    meta['expected_player_answers'] = np.where(meta.team_answered, meta.normalized_pred, 0)
    return meta.expected_player_answers.values

In [39]:
def m_step(model, X, train_y, expected_player_probas):
    model.fit(X, train_y, sample_weight=expected_player_probas)
    return model

In [40]:
weights = e_step(model, train_X_csr, train_meta)

In [41]:
train_meta.shape, weights.shape, train_X_csr.shape

((17852183, 3), (17852183,), (17852183, 90810))

In [42]:
def get_team_rate(model, players):
    player_weights = model.coef_[question_max_token + 1:]
    players_rating = {i:rating for i, rating in enumerate(player_weights)}
    rates = np.array([players_rating[player_id] for player_id in players])
    return 1 - np.prod(1 - rates)

def get_scores(df, model):
    spearman_scores = []
    kendall_scores = []
    for _, tournament_df in df.groupby('tournament_id'):
        true_scores = []
        pred_scores = []
        for _, team_df in tournament_df.groupby('team_id'):
            team_players = team_df.player_new_id.values
            team_true_score = team_df.questionsTotal.iloc[0]
            team_pred_score = get_team_rate(model, team_players)
            true_scores.append(team_true_score)
            pred_scores.append(team_pred_score)
        spearman_scores.append(spearmanr(true_scores, pred_scores).correlation)
        kendall_scores.append(kendalltau(true_scores, pred_scores).correlation)
    return np.array(spearman_scores), np.array(kendall_scores)

In [43]:
model = LinearRegression(fit_intercept=False)
model.fit(train_X_csr, train_y)
for i in range(5):
    print(f'Step {i}')
    expected_player_probas = e_step(model, train_X_csr, train_meta)
    model = m_step(model, train_X_csr, train_y, expected_player_probas)
    spearman, kendall = get_scores(train, model)
    print(spearman[~np.isnan(spearman)].mean(), kendall[~np.isnan(kendall)].mean())

Step 0


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


0.8252373429417303 0.6728593890833944
Step 1


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


0.8257611934147547 0.673262457992201
Step 2


...

0.8257495803482544 0.6732371021752803
Step 3


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


0.8257493850729275 0.6732369280553763
Step 4


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


0.8257495299377987 0.6732370157087451


## Пункт 5

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [44]:
question_ratings = model.coef_[:question_max_token+1]

In [45]:
tournament_diff = {}
i = 0
for tournament_new_id in train.tournament_new_id.unique():
    question_diff = []
    for _ in range(train[train['tournament_new_id'] == tournament_new_id].mask_len.max()):
        question_diff.append(question_ratings[i-1])
        i += 1
    tournament_diff[tournament_new_id] = np.mean(question_diff)

In [46]:
train['tournament_rating'] = train['tournament_new_id'].map(tournament_diff)

In [47]:
tournament_rating_df = train.drop_duplicates(subset=['tournament_id']).sort_values(
    by='tournament_rating', ascending=True)[['tournament_id', 'tournament_rating']]

In [48]:
tournament_rating_df = pd.merge(tournament_rating_df, train_tournaments_df[['id', 'name']], left_on='tournament_id', right_on='id', how='inner')

In [49]:
tournament_rating_df.head()

,tournament_id,tournament_rating,id,name
0,5928,-0.045036,5928,Угрюмый Ёрш
1,5159,-0.029176,5159,Первенство правого полушария
2,5684,-0.024599,5684,Синхрон высшей лиги Москвы
3,5827,-0.010098,5827,Шестой киевский марафон. Асинхрон
4,5943,-0.005813,5943,Чемпионат Мира. Этап 2 Группа С


In [50]:
tournament_rating_df.tail()

,tournament_id,tournament_rating,id,name
670,5009,0.633034,5009,(а)Синхрон-lite. Лига старта. Эпизод III
671,5012,0.652456,5012,Школьный Синхрон-lite. Выпуск 2.5
672,5936,0.659953,5936,Школьная лига. I тур.
673,5013,0.678249,5013,(а)Синхрон-lite. Лига старта. Эпизод V
674,5955,0.680198,5955,Школьная лига. III тур.


### Рейтинг игроков

In [51]:
player_weights = model.coef_[question_max_token + 1:]

players_rating = {i:rating for i, rating in enumerate(player_weights)}

players_rating_df = train[['player_name', 'player_new_id']].drop_duplicates(subset=['player_new_id'])

players_rating_df['rating'] = players_rating_df['player_new_id'].map(players_rating)

players_rating_df = players_rating_df.sort_values(by='rating', ascending=False)

players_rating_df = players_rating_df.reset_index(drop=True)

In [52]:
players_rating_df.head(20)

,player_name,player_new_id,rating
0,Ушаков Илья,34927,1.041344
1,Койфман Лия,42428,1.036816
2,Гуменюк Мария,55899,0.833004
3,Бурак Ирина,122,0.715381
4,Саранцев Алексей,131,0.708169
5,Мирзоев Анар,25375,0.703997
6,Немец Илья,33171,0.694647
7,Соловьёва Людмила,32527,0.679973
8,Александрова Елена,24135,0.673719
9,Микава Этери,33654,0.664549


Рейтинги игроков рассмотренных для бейзлайна. Видно, что модель стала лучше не только на скорах, но и на конкретных примерах

In [53]:
players_rating_df[players_rating_df['player_name'] == 'Николенко Сергей']

,player_name,player_new_id,rating
10404,Николенко Сергей,3,0.189268


In [54]:
players_rating_df[players_rating_df['player_name'] == 'Сорожкин Артём']

,player_name,player_new_id,rating
57,Сорожкин Артём,155,0.536829


In [55]:
players_rating_df[players_rating_df['player_name'] == 'Савченков Михаил']

,player_name,player_new_id,rating
27,Савченков Михаил,1208,0.575251


In [56]:
players_rating_df[players_rating_df['player_name'] == 'Левандовский Михаил']

,player_name,player_new_id,rating
97,Левандовский Михаил,2,0.512897


In [57]:
players_rating_df[players_rating_df['player_name'] == 'Брутер Александра']

,player_name,player_new_id,rating
28,Брутер Александра,5957,0.57378


## Вывод

Изначально задача казалось нерешаемой. Единственной предположение было посчитать число правильных ответов совместно команды и попробовать как-то взвесить турниры. Но хорошо что в описании было сказано как подступиться и получили какие-то результаты. Думаю моя ЕМ схема составлена не лучшим образом, так как подозрительно слишком быстро она сошлась к чему-то :) <br>